# **Préparation et nettoyage des données loyers**

---

## **Introduction**

### **Objectif de ce notebook**

Ce notebook a pour objectif de préparer les données loyers.

### **Sources de données utilisées**

**"Carte des loyers" - Indicateurs de loyers d'annonce par commune en 2024, [source](https://www.data.gouv.fr/datasets/carte-des-loyers-indicateurs-de-loyers-dannonce-par-commune-en-2024/)**
- développée par la Direction Générale de l'Aménagement, du Logement et de la Nature,
- basée sur 9,9 millions d'annonces locatives, permettant d'estimer les prix des loyers par commune pour le 3ème trimestre 2024.

Les indicateurs sont calculés à partir des données de leboncoin et SeLoger; ils couvrent toute la France et concernent différents types de logements.

Le document comporte également d'importantes mises en garde concernant l'utilisation de ces estimations, recommandant la prudence pour :
- Les communes avec peu d'observations (moins de 30)
- Les zones avec un faible coefficient de détermination (R2 < 0,5)
- Les zones avec des intervalles de prédiction très larges

**Liste des données**
- [Loyer appartement](https://static.data.gouv.fr/resources/carte-des-loyers-indicateurs-de-loyers-dannonce-par-commune-en-2024/20241205-153050/pred-app-mef-dhup.csv)
- [Loyer appartement T1-T2](https://static.data.gouv.fr/resources/carte-des-loyers-indicateurs-de-loyers-dannonce-par-commune-en-2024/20241205-153048/pred-app12-mef-dhup.csv)
- [Loyer appartement T3+](https://static.data.gouv.fr/resources/carte-des-loyers-indicateurs-de-loyers-dannonce-par-commune-en-2024/20241205-145658/pred-app3-mef-dhup.csv)
- [Loyer maison](https://static.data.gouv.fr/resources/carte-des-loyers-indicateurs-de-loyers-dannonce-par-commune-en-2024/20241205-145700/pred-mai-mef-dhup.csv)

---

## **Import des bibliothèques**


In [1]:
# Manipulation de données
import pandas as pd
import numpy as np
import re
from typing import Tuple, List, Dict, List, Optional, Union

# Visualisation
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Utilitaires
from datetime import datetime
from rapidfuzz import process, fuzz

---

## **Chargement des données brutes**

### **Chargement et fusionnement des fichiers de loyers**

In [ ]:
def load_data(file_path, name):
    df = pd.read_csv(file_path,  encoding='latin1', sep=';')
    # Add source column
    df['type_local'] = name
    return df

def compare_columns(df1, df2, df3, df4):
    print("Compare columns:")
    print(f"Number of columns: {len(df1.columns)} == {len(df2.columns)} == {len(df3.columns)} == {len(df4.columns)}")

    for col in df1.columns:
        in_df2 = col in df2.columns
        in_df3 = col in df3.columns
        in_df4 = col in df4.columns
        if in_df2 and in_df3 and in_df4:
            print(f"   - {col} OK in all df")
        else:
            print(f"   - {col} MISSING in: ", end="")
            if not in_df2:
                print("df2 ", end="")
            if not in_df3:
                print("df3 ", end="")
            if not in_df4:
                print("df4 ", end="")
    return

def merge_datasets(df1, df2, df3, df4):
    merged_df = pd.concat([df1, df2, df3, df4], ignore_index=True)
    return merged_df

df_appartements = load_data("data/raw/pred-app-mef-dhup.csv", "Appartement")
df_appartements3plus = load_data("data/raw/pred-app3-mef-dhup.csv", "Appartement T3+")
df_appartements12 = load_data("data/raw/pred-app12-mef-dhup.csv", "Appartement T1-T2")
df_maisons = load_data("data/raw/pred-mai-mef-dhup.csv", "Maison")
# compare_columns(df_appartements, df_appartements3plus, df_appartements12, df_maisons)

merged_df = merge_datasets(df_appartements, df_appartements3plus, df_appartements12, df_maisons)

### **Chargement des fichiers supplémentaires**

In [3]:
communes_df = pd.read_csv("data/cleaned/communes.csv")
communes_df.drop(columns="code_departement", inplace=True)


---

## **Exploration initiale des données**

### **Structure du dataset**


In [4]:
def get_summary(df, show_missing=False):
    """Get a summary of the current dataset state"""
    print("="*100)
    print("RÉSUMÉ DU JEU DE DONNÉES")
    print("="*100)
    print(f"Dimensions : {df.shape[0]} lignes × {df.shape[1]} colonnes")
    print(f"\nTypes de données :")
    print(df.dtypes.value_counts())
    if show_missing:
        print(f"\nValeurs manquantes :")
        missing = df.isnull().sum()
        missing = missing[missing > 0].sort_values(ascending=False)
        if len(missing) > 0:
            for col, count in missing.items():
                pct = (count / len(df)) * 100
                print(f"  {col}: {count} ({pct:.1f}%)")
        else:
            print("  Aucune valeur manquante !")

get_summary(merged_df, show_missing=True)

RÉSUMÉ DU JEU DE DONNÉES
Dimensions : 139840 lignes × 14 colonnes

Types de données :
object    11
int64      3
Name: count, dtype: int64

Valeurs manquantes :
  Aucune valeur manquante !


#### **Analyse des valeurs manquantes**
Il n'y a aucune valeur manquante dans ces ensembles de données.

---

### **Analyse des colonnes - Colonnes de la Carte des Loyers 2024**

- `id_zone` : Identifiant unique de la zone
- `INSEE_C` : Code INSEE de la commune
- `LIBGEO` --> `nom_commune` : Nom de la zone géographique (commune)
- `EPCI` : Code de groupement intercommunal
- `DEP` : Code du département
- `REG` : Code de la région
- `loypredm2` --> `loyer_predit_m2` : Prix de location prédit par m²
- `lwr.IPm2` --> `borne_inferieure_intervalle` : Borne inférieure de l'intervalle de prédiction
- `upr.IPm2` --> `borne_superieure_intervalle` : Borne supérieure de l'intervalle de prédiction
- `TYPPRED` --> `type_prediction` : Type de prédiction ("commune" ou "maille")
- `nbobs_com` --> `nombre_observations_commune` : Nombre d'observations dans la commune
- `nbobs_mail` --> `nombre_observations_zone` : Nombre d'observations dans la zone plus large
- `R2_adj` --> `coefficient_determination_ajuste` : R² ajusté (mesure statistique de l'ajustement)
- `type_local` : Type de bien (source du CSV avant la fusion)

#### **Précautions d'utilisation**
Être prudent avec les données où :
- coefficient_determination_ajuste < 0,5
- nombre_observations_commune < 30
- L'intervalle de prédiction est très large

In [5]:
merged_df.columns

Index(['id_zone', 'INSEE_C', 'LIBGEO', 'EPCI', 'DEP', 'REG', 'loypredm2',
       'lwr.IPm2', 'upr.IPm2', 'TYPPRED', 'nbobs_com', 'nbobs_mail', 'R2_adj',
       'type_local'],
      dtype='object')

In [6]:
column_mapping = {
    'INSEE_C': 'INSEE_COM',
    'LIBGEO': 'nom_commune',
    'loypredm2': 'loyer_predit_m2',
    'lwr.IPm2': 'borne_inferieure_intervalle',
    'upr.IPm2': 'borne_superieure_intervalle',
    'TYPPRED': 'type_prediction',
    'nbobs_com': 'nombre_observations_commune',
    'nbobs_mail': 'nombre_observations_zone',
    'R2_adj': 'coefficient_determination_ajuste'
}
merged_df = merged_df.rename(columns=column_mapping)

In [7]:
def analyse_and_print_columns_compact(df):
    """ Print column analysis in a compact format """
    print("="*100)
    print("ANALYSE DES COLONNES")
    print("="*100)
    print(f"Nombre total de colonnes : {len(df.columns)}")
    print(f"\nListe des colonnes : {', '.join(df.columns)}")
    print("\n")
    for column in df.columns:
        print("-" * 2*len(column))
        print(f"{column}")
        print("-" * 2*len(column))
        print(f"Type: {df[column].dtype}")
        print(f"Valeurs nulles : {df[column].isnull().sum()}")
        print(f"Valeurs uniques : {df[column].nunique()}")
        print("Valeurs principales :")
        print(df[column].value_counts().head().to_string())
        print("\n")

analyse_and_print_columns_compact(merged_df)

ANALYSE DES COLONNES
Nombre total de colonnes : 14

Liste des colonnes : id_zone, INSEE_COM, nom_commune, EPCI, DEP, REG, loyer_predit_m2, borne_inferieure_intervalle, borne_superieure_intervalle, type_prediction, nombre_observations_commune, nombre_observations_zone, coefficient_determination_ajuste, type_local


--------------
id_zone
--------------
Type: object
Valeurs nulles : 0
Valeurs uniques : 3216
Valeurs principales :
id_zone
453    216
90     202
193    191
367    191
200    182


------------------
INSEE_COM
------------------
Type: object
Valeurs nulles : 0
Valeurs uniques : 34960
Valeurs principales :
INSEE_COM
86051    4
16136    4
16002    4
79150    4
16104    4


----------------------
nom_commune
----------------------
Type: object
Valeurs nulles : 0
Valeurs uniques : 32713
Valeurs principales :
nom_commune
Sainte-Colombe    48
Saint-Sauveur     44
Saint-Aubin       40
Beaulieu          40
Sainte-Marie      36


--------
EPCI
--------
Type: object
Valeurs nulles : 0
V

#### **Analyse des colonnes**
D'après l'analyse, nous constatons que les colonnes suivantes nécessiteront un changement de type :
- `loyer_predit_m2` --> numérique
- `borne_inferieure_intervalle` --> numérique
- `borne_superieure_intervalle` --> numérique
- `coefficient_determination_ajuste` --> numérique
- `REG` --> catégoriel

---

## **Nettoyage des données**

### **Standardisation des noms de colonnes**

In [8]:
def clean_col_name(col: str) -> str:
    """Internal helper to clean a single column name."""
    cleaned_col = "".join(char if char.isprintable() else ' ' for char in col)
    cleaned_col = re.sub(r'\s+', ' ', cleaned_col)
    return cleaned_col.strip()

def find_unprintable_columns(df):
    """Find columns with unprintable characters or whitespace issues in names"""
    issues = []
    for col in df.columns:
        problems = []
        # Check for unprintable characters
        if not col.isprintable():
            problems.append("unprintable characters")
        # Check for leading/trailing whitespace
        if col != col.strip():
            problems.append("leading/trailing whitespace")
        # Check for internal multiple spaces
        if '  ' in col:
            problems.append("multiple internal spaces")
        # Check for tabs
        if '\t' in col:
            problems.append("tab characters")
        # Check for newlines
        if '\n' in col or '\r' in col:
            problems.append("newline characters")
        if problems:
            issues.append({
                'column': repr(col),
                'issues': ', '.join(problems),
                'cleaned_version': clean_col_name(col)
            })
    if issues:
        print(f"Trouvé {len(issues)} colonnes avec des problèmes :")
        for item in issues:
            print(f"  Colonne : {item['column']}")
            print(f"    Problèmes : {item['issues']}")
            print(f"    Deviendra : '{item['cleaned_version']}'")
    else:
        print("✓ Tous les noms de colonnes sont propres !")
    return issues

def standardise_column_names(df):
    """Standardise column names by removing control characters like \\n, \\t"""
    old_cols = df.columns.tolist()
    df.columns = [clean_col_name(col) for col in df.columns]
    changed = sum(1 for old, new in zip(old_cols, df.columns) if old != new)
    print(f"✓ Nettoyé {changed} noms de colonnes")
    return df
    
issues = find_unprintable_columns(merged_df)

if issues:
    merged_df = standardise_column_names(merged_df)

✓ Tous les noms de colonnes sont propres !


### **Sélection des colonnes pertinentes**

On commence en filtrant sur la région pertinante.

**Colonnes essentielles**
- `nom_commune` : Nom de la commune (pour visualisation)
- `INSEE_COM` : Code INSEE (pour jointures)
- `DEP` : Code département
- `loyer_predit_m2` : Prix de location prédit par m²

**Colonnes pour l'analyse de qualité**
- `nombre_observations_commune` : Nombre d'observations dans la commune
- `coefficient_determination_ajuste` : Qualité statistique de la prédiction
- `type_prediction` : Type de prédiction ("commune" ou "maille")

**Colonnes d'intervalles**
- `borne_inferieure_intervalle` : Borne inférieure de l'estimation
- `borne_superieure_intervalle` : Borne supérieure de l'estimation

In [9]:
columns_to_drop = ["id_zone", "EPCI"]

def drop_columns(df):
    """Drop columns that are of no interest"""
    dropped = []
    for col in df.columns:
        if col in columns_to_drop:
            df = df.drop(columns=[col])
            dropped.append(col)
    if len(dropped) > 0:
        print(f"✓ Colonnes supprimées : {', '.join(dropped)}")
    else:
        print("✓ Aucune colonne à supprimer")
    return df

merged_df = drop_columns(merged_df)

✓ Colonnes supprimées : id_zone, EPCI


### **Conversion des types de données**

In [10]:
numeric_columns = ["loyer_predit_m2", "borne_inferieure_intervalle", "borne_superieure_intervalle", "coefficient_determination_ajuste"]
categorial_columns = ["REG"]

def convert_to_numeric(df, numeric_columns=numeric_columns):
    """Converts specified columns to numeric type"""
    converted = []
    for col in numeric_columns:
        if col not in df.columns:
            print(f"⚠ Colonne '{col}' non trouvée, ignorer")
            continue
        
        try:
            df[col] = pd.to_numeric(
                df[col].astype(str).str.replace(',', '.', regex=False), 
                errors='coerce'
            )
            converted.append(col)
        except Exception as e:
            print(f"⚠ Erreur de conversion de '{col}' : {e}")
    
    print(f"✓ Converti {len(converted)} colonnes en type numérique")
    return df

def convert_to_categorial(df, categorial_columns=categorial_columns):
    """Converts specified columns to categorial type"""
    converted = []
    for col in categorial_columns:
        if col not in df.columns:
            print(f"⚠ Colonne '{col}' non trouvée, ignorer")
            continue
        
        try:
            df[col] = df[col].astype(str).str.strip()
            df[col] = df[col].astype('object')
            converted.append(col)
        except Exception as e:
            print(f"⚠ Erreur de conversion de '{col}' : {e}")
    
    print(f"✓ Converti {len(converted)} colonnes en type catégoriel")
    return df

merged_df = convert_to_numeric(merged_df)
merged_df = convert_to_categorial(merged_df)

# Into categorial, so we can merge later on
communes_df = convert_to_categorial(communes_df, ["code_commune_INSEE"])

✓ Converti 4 colonnes en type numérique
✓ Converti 1 colonnes en type catégoriel
✓ Converti 1 colonnes en type catégoriel


### **Suppression des espaces superflus**

In [11]:
def find_whitespace_in_values(df):
    """Find columns with leading/trailing whitespace in values"""
    whitespace_info = []
    string_cols = df.select_dtypes(include=['object', 'string']).columns
    for col in string_cols:
        has_whitespace = df[col].astype(str).str.strip() != df[col].astype(str)
        if has_whitespace.any():
            count = has_whitespace.sum()
            whitespace_mask = has_whitespace
            examples_original = df[col][whitespace_mask].head(3).tolist()
            examples_cleaned = [str(val).strip() for val in examples_original]
            examples_orig_str = ' | '.join([f'"{val}"' for val in examples_original])
            examples_clean_str = ' | '.join([f'"{val}"' for val in examples_cleaned])
            whitespace_info.append({
                'column': col,
                'affected_rows': count,
                'percentage': round(count / len(df) * 100, 2),
                'examples_before': examples_orig_str,
                'examples_after': examples_clean_str
            })
    return pd.DataFrame(whitespace_info)

def trim_whitespace(df, columns=None):
    """Trim whitespace from specified columns (or all string columns if None)"""
    string_cols = df.select_dtypes(include=['object', 'string']).columns
    if columns is not None:
        string_cols = [col for col in columns if col in string_cols]
    
    for col in string_cols:
        df[col] = df[col].str.strip()
    
    print(f"✓ Supprimé les espaces en début/fin de {len(string_cols)} colonnes")
    return df

whitespace_df = find_whitespace_in_values(merged_df)

if len(whitespace_df) > 0:
    print("Colonnes avec problèmes d'espaces :")
    display(whitespace_df)
    
    # Trim whitespace from all string columns
    trim_whitespace(merged_df)
else:
    print("✓ Aucun problème d'espaces trouvé !")

✓ Aucun problème d'espaces trouvé !


### **Harmonisation des majuscules/minuscules**

In [12]:
def find_case_insensitive_duplicates(df):
    """
    Finds columns with case-insensitive duplicates (e.g., 'Apple', 'apple').
    Returns a DataFrame summarizing the issues for easy display.
    """
    results = []
    string_cols = df.select_dtypes(include=['object', 'string']).columns
    for col in string_cols:
        series = df[col]
        clean_series = series.dropna().astype(str)
        if len(clean_series) == 0:
            continue
        case_map = {}
        for value in clean_series.unique():
            lower_val = value.lower()
            if lower_val in case_map:
                case_map[lower_val].append(value)
            else:
                case_map[lower_val] = [value]
        duplicate_groups = [group for group in case_map.values() if len(group) > 1]
        if duplicate_groups:
            value_counts = df[col].value_counts()
            total_affected_rows = 0
            example_groups = []
            for group in duplicate_groups:
                total_affected_rows += value_counts[group].sum()
                most_frequent_form = max(group, key=lambda x: value_counts.get(x, 0))
                group_str = ' | '.join([f'"{val}"' for val in sorted(group)])
                example_groups.append(f'{group_str} -> "{most_frequent_form}"')
            results.append({
                'column': col,
                'duplicate_groups': len(duplicate_groups),
                'affected_rows': total_affected_rows,
                'examples': ' || '.join(example_groups[:3])
            })
    return pd.DataFrame(results)

def standardise_case(df, columns: list):
    """
    Standardises the casing of values in the specified columns.
    """
    standardised_count = 0
    for col in columns:
        if col not in df.columns:
            continue
        series = df[col]
        clean_series = series.dropna().astype(str)
        if len(clean_series) == 0:
            continue
        case_map = {}
        for value in clean_series.unique():
            lower_val = value.lower()
            if lower_val in case_map:
                case_map[lower_val].append(value)
            else:
                case_map[lower_val] = [value]
        duplicate_groups = [group for group in case_map.values() if len(group) > 1]
        if not duplicate_groups:
            continue
        value_counts = df[col].value_counts()
        replacement_map = {}
        for group in duplicate_groups:
            most_frequent_form = max(group, key=lambda x: value_counts.get(x, 0))
            for variant in group:
                if variant != most_frequent_form:
                    replacement_map[variant] = most_frequent_form
        if replacement_map:
            df[col] = df[col].replace(replacement_map)
            standardised_count += 1
    
    print(f"✓ Standardisé la casse dans {standardised_count} colonnes")
    return df

case_dups_df = find_case_insensitive_duplicates(merged_df)

if len(case_dups_df) > 0:
    print("Colonnes avec des doublons insensibles à la casse :")
    display(case_dups_df)
    
    # Standardize case for affected columns
    columns_to_standardise = case_dups_df['column'].tolist()
    standardise_case(columns_to_standardise)
else:
    print("✓ Aucun doublon insensible à la casse trouvé !")

✓ Aucun doublon insensible à la casse trouvé !


### **Trouver des groupes de chaînes similaires (potentielles erreurs de frappe) dans les colonnes catégorielles**

In [13]:
def normalise_for_comparison(s: str) -> str:
    """Intelligently cleans a string for a base similarity comparison."""
    if not isinstance(s, str):
        return ""
    s_lower = s.lower()
    s_lower = re.sub(r'tbc\s*\(proposition\s*-?|local\s*|à\s*confirmer|pp\s*\d', '', s_lower)
    s_lower = re.sub(r'[\s-]+', '', s_lower)
    s_lower = s_lower.strip("()[]{}'\"- ")
    return s_lower

def find_fuzzy_duplicates(df, threshold: int = 85, min_length: int = 3):
    """
    Finds groups of similar strings (potential typos) in categorical columns.
    """
    issue_list = []
    string_cols = df.select_dtypes(include=['object', 'string']).columns
    for col in string_cols:
        series = df[col]
        if series.nunique() < 2 or series.nunique() > 2000:
            continue
        categories = series.dropna().unique().tolist()
        filtered_cats = [
            cat for cat in set(categories)
            if isinstance(cat, str) and len(cat) >= min_length and not re.search(r'\d', cat)
        ]
        if len(filtered_cats) < 2:
            continue
        normalised_cats = [normalise_for_comparison(cat) for cat in filtered_cats]
        score_matrix = process.cdist(normalised_cats, normalised_cats, scorer=fuzz.ratio, score_cutoff=threshold)
        groups = []
        processed_indices = set()
        for i in range(len(filtered_cats)):
            if i in processed_indices:
                continue
            nonzero_result = score_matrix[i].nonzero()
            if isinstance(nonzero_result, tuple) and len(nonzero_result) > 0:
                similar_indices = nonzero_result[0] if len(nonzero_result) == 1 else nonzero_result[1]
            else:
                continue
            if len(similar_indices) > 1:
                current_group = {filtered_cats[j] for j in similar_indices}
                groups.append(sorted(list(current_group)))
                processed_indices.update(similar_indices)
        if groups:
            issue_list.append({'column': col, 'fuzzy_groups': groups})
    return issue_list

fuzzy_issues = find_fuzzy_duplicates(merged_df, threshold=85, min_length=3)

if fuzzy_issues:
    print(f"Trouvé des doublons approximatifs dans {len(fuzzy_issues)} colonnes :\n")
    for issue in fuzzy_issues:
        print(f"Colonne : {issue['column']}")
        for i, group in enumerate(issue['fuzzy_groups'], 1):
            print(f"  Groupe {i}: {group}")
        print()
else:
    print("✓ Aucun doublon approximatif trouvé !")

✓ Aucun doublon approximatif trouvé !


### **Suppression des doublons**

In [14]:
def remove_duplicates(df):
    """Removes duplicate rows from the current DataFrame."""
    initial_rows = len(df)
    df = df.drop_duplicates(ignore_index=True)
    removed_rows = initial_rows - len(df)
    print(f"✓ Supprimé {removed_rows} lignes en double (conservé {len(df)} lignes uniques)")
    return df

merged_df = remove_duplicates(merged_df)

✓ Supprimé 0 lignes en double (conservé 139840 lignes uniques)


### **Filtrage des données**
Selon deux critères principaux :

**Filtrage géographique pour l'Île-de-France**
- Sélection des communes de la région Île-de-France
- Codes région : REG = "11"
- Codes départements : 75 (Paris), 77, 78, 91, 92, 93, 94, 95

In [15]:
def filter_region(df):
    """Filter for Ile-de-France region."""
    print("="*100)
    print("FILTRER POUR L'ÎLE-DE-FRANCE")
    print("="*100)
    filtered_df = df[df["REG"] == "11"]
    print(f"Taille du jeu de données initial : {len(df)}")
    print(f"Taille du jeu de données IDF : {len(filtered_df)}")
    departments = sorted(filtered_df["DEP"].unique().tolist())
    print(f"\nDépartements en IDF : {', '.join(departments)}")
    types = filtered_df["type_local"].unique().tolist()
    print(f"\nTypes de biens : {', '.join(types)}")
    return filtered_df

departments_dict = {
    # Île-de-France (Région 11)
    '75': 'Paris',
    '77': 'Seine-et-Marne',
    '78': 'Yvelines',
    '91': 'Essonne',
    '92': 'Hauts-de-Seine',
    '93': 'Seine-Saint-Denis',
    '94': 'Val-de-Marne',
    '95': 'Val-d\'Oise'
}

def get_department_name(code):
    return departments_dict.get(code, f"Département {code}")

idf_data = merged_df.copy()
idf_data = filter_region(idf_data)

merged_df = merged_df.rename(columns=column_mapping)

idf_data['nom_departement'] = idf_data['DEP'].map(get_department_name)

FILTRER POUR L'ÎLE-DE-FRANCE
Taille du jeu de données initial : 139840
Taille du jeu de données IDF : 5144

Départements en IDF : 75, 77, 78, 91, 92, 93, 94, 95

Types de biens : Appartement, Appartement T3+, Appartement T1-T2, Maison


**Filtrage qualitatif des données selon ANIL**
- Nombre minimal d'observations : nombre_observations_commune ≥ 30
- Qualité statistique : coefficient_determination_ajuste ≥ 0,5
- Type de prédiction : type_prediction = "commune"

In [16]:
def analyse_outliers(df):
    """Analyse outliers defined by the dataset owners."""
    print("="*100)
    print("ANALYSE DES VALEURS ABERRANTES")
    print("Filtrage selon les critères définis par ANIL")
    print("="*100)
    
    # Low observation count
    low_obs = df[df['nombre_observations_commune'] < 30]
    print(f"Communes avec moins de 30 observations : {len(low_obs)} ({len(low_obs)/len(df)*100:.2f}%)")
    print("Échantillon de communes à faible observation :")
    print(low_obs[['nom_commune', 'DEP', 'nombre_observations_commune']].head())
    
    # Low R2 adjustment
    low_r2 = df[df['coefficient_determination_ajuste'] < 0.5]
    print(f"\nCommunes avec coefficient_determination_ajuste < 0.5 : {len(low_r2)} ({len(low_r2)/len(df)*100:.2f}%)")
    print("Échantillon de communes à faible R2 :")
    print(low_r2[['nom_commune', 'DEP', 'coefficient_determination_ajuste']].head())
    
    # Non-commune predictions
    non_commune = df[df['type_prediction'] != 'commune']
    print(f"\nPrédictions non communales : {len(non_commune)} ({len(non_commune)/len(df)*100:.2f}%)")
    print("Échantillon de prédictions non communales :")
    print(non_commune[['nom_commune', 'DEP', 'type_prediction']].head())

    # Create a summary mask
    mask = (
        (df['nombre_observations_commune'] >= 10) &
        (df['coefficient_determination_ajuste'] >= 0.5)
        & (df['type_prediction'] == 'commune')
    )
    filtered_df = df[mask]
    
    print(f"\nTaille du jeu de données initial : {len(df)}")
    print(f"Perte de données selon les recommandations de l'ANIL : {len(df) - len(filtered_df)} données ({(1 - len(filtered_df)/len(df))*100:.2f}% du jeu de données initial)")
    print(f"Taille du jeu de données filtré : {len(filtered_df)} ({len(filtered_df)/len(df)*100:.2f}%)")

analyse_outliers(idf_data)

ANALYSE DES VALEURS ABERRANTES
Filtrage selon les critères définis par ANIL
Communes avec moins de 30 observations : 2517 (48.93%)
Échantillon de communes à faible observation :
         nom_commune DEP  nombre_observations_commune
361        Villebéon  77                            1
362      Chaintreaux  77                            2
364         Bransles  77                           12
367       Remauville  77                            5
368  Vaux-sur-Lunain  77                            0

Communes avec coefficient_determination_ajuste < 0.5 : 30 (0.58%)
Échantillon de communes à faible R2 :
              nom_commune DEP  coefficient_determination_ajuste
40930              Lisses  91                          0.219446
40931  Évry-Courcouronnes  91                          0.219446
41461    Épinay-sur-Seine  93                          0.396006
41462           Montmagny  95                          0.396006
44708        La Courneuve  93                          0.415650

Prédicti

Étant donné qu'on perdra plus de 60% de nos données en suivant les recommandations de l'ANIL, on a décidé d'adapter nos critères de filtrage de manière plus souple.

**Filtrage qualitatif des données**
- Nombre minimal d'observations : nombre_observations_commune ≥ 10
- Qualité statistique : coefficient_determination_ajuste ≥ 0,5
- Type de prédiction : nous ne filtrerons pas sur ce critère pour maximiser la couverture des données

In [17]:
def filter_outliers(df):
    """Filter out outliers."""
    print("="*100)
    print("FILTRAGE DES VALEURS ABERRANTES")
    print("="*100)

    mask = (
        (df['nombre_observations_commune'] >= 10) &
        (df['coefficient_determination_ajuste'] >= 0.5)
        # & (df['type_prediction'] == 'commune')
    )
    filtered_df = df[mask]
    
    print(f"Taille du jeu de données initial : {len(df)}")
    print(f"Taille du jeu de données filtré : {len(filtered_df)} ({len(filtered_df)/len(df)*100:.2f}%)")
    
    departments = sorted(filtered_df["DEP"].unique().tolist())
    if departments:
        print(f"\nDépartements dans le jeu de données filtré : {', '.join(departments)}")
    
    types = filtered_df["type_local"].unique().tolist()
    if types:
        print(f"\nTypes de biens : {', '.join(types)}")
    return filtered_df

filtered_df = filter_outliers(idf_data)

FILTRAGE DES VALEURS ABERRANTES
Taille du jeu de données initial : 5144
Taille du jeu de données filtré : 3548 (68.97%)

Départements dans le jeu de données filtré : 75, 77, 78, 91, 92, 93, 94, 95

Types de biens : Appartement, Appartement T3+, Appartement T1-T2, Maison


---

## Analyses statistiques descriptives

### Statistiques globales

In [18]:
numeric_columns = filtered_df.select_dtypes(include=[np.number]).columns

def numeric_column_stats(df, numeric_columns=numeric_columns):
    print("="*100)
    print("STATISTIQUES DESCRIPTIVES POUR LES COLONNES NUMÉRIQUES")
    print("="*100)

    num_stats = df[numeric_columns].describe().T
    display(num_stats)

numeric_column_stats(filtered_df)

STATISTIQUES DESCRIPTIVES POUR LES COLONNES NUMÉRIQUES


,count,mean,std,min,25%,50%,75%,max
loyer_predit_m2,3548.0,16.810087,4.294699,7.533031,14.033683,16.172932,18.800197,39.415339
borne_inferieure_intervalle,3548.0,13.737271,3.462850,5.609561,11.308727,13.322453,15.589108,30.084113
borne_superieure_intervalle,3548.0,20.690876,6.046823,9.503648,17.000975,19.541782,22.685796,57.666241
nombre_observations_commune,3548.0,786.650507,2150.009362,10.000000,27.000000,87.000000,506.500000,40020.000000
nombre_observations_zone,3548.0,1473.955750,2169.904894,468.000000,528.000000,740.000000,1503.250000,40020.000000
coefficient_determination_ajuste,3548.0,0.785770,0.086958,0.502456,0.729719,0.802542,0.851530,0.969416


### **Distribution des variables clés**
#### **Répartition par département et type de prédiction**

In [19]:
def categorial_distributions(df):
    print("="*100)
    print("STATISTIQUES DESCRIPTIVES POUR LES COLONNES CATÉGORIELLES")
    print("="*100)

    columns_to_plot = ["DEP", "type_prediction"]

    fig = make_subplots(
        rows=1,
        cols=2,
        subplot_titles=[f"Distribution de {col}" for col in columns_to_plot]
    )

    for i, col in enumerate(columns_to_plot):
        value_counts = df[col].value_counts()

        # Position in subplot
        row = i // 2 + 1
        col_pos = i % 2 + 1
        if col == "DEP":
            bar = go.Bar(
                x=value_counts.index.astype(str),
                y=value_counts.values,
                name=col,
                text=value_counts.values,
                hovertemplate='Département: %{x} (%{customdata[1]})<br>Nombre de bien: %{y} (%{customdata[0]:.2f}%)<extra></extra>',
                customdata=list(zip(
                    [(value / len(df) * 100) for value in value_counts.values],
                    value_counts.index.map(get_department_name)
                ))
            )
        else:
            bar = go.Bar(
                x=value_counts.index.astype(str),
                y=value_counts.values,
                name=col,
                text=value_counts.values,
                hovertemplate='Type de prédiction: %{x}<br>Nombre de bien: %{y} (%{customdata:.2f}%)<extra></extra>',
                customdata=[(value / len(df) * 100) for value in value_counts.values]
            )
        fig.add_trace(bar, row=row, col=col_pos)

    fig.update_layout(title_text="Distributions des colonnes catégorielles", showlegend=False)

    fig.update_xaxes(tickangle=45)

    fig.show()

categorial_distributions(filtered_df)

STATISTIQUES DESCRIPTIVES POUR LES COLONNES CATÉGORIELLES


#### **Distribution des biens par communes**

In [20]:
def analyse_distribution_communes(df):
    print("="*100)
    print("STATISTIQUES DE DISTRIBUTION DES COMMUNES")
    print("="*100)
    
    communes_counts = df['nom_commune'].value_counts()
    
    print(f"Nombre total de communes : {len(communes_counts)}")
    print(f"Nombre moyen de biens par commune : {communes_counts.mean():.2f}")
    print(f"Médiane de biens par commune : {communes_counts.median():.2f}\n")
    
    print("="*100)
    print("RÉPARTITION DES COMMUNES PAR NOMBRE DE BIENS")
    print("="*100)
    unique_counts = communes_counts.value_counts().sort_index()
    for count, num_communes in unique_counts.items():
        print(f"{num_communes} communes ont {count} bien(s)")

analyse_distribution_communes(filtered_df)

STATISTIQUES DE DISTRIBUTION DES COMMUNES
Nombre total de communes : 1073
Nombre moyen de biens par commune : 3.31
Médiane de biens par commune : 4.00

RÉPARTITION DES COMMUNES PAR NOMBRE DE BIENS
130 communes ont 1 bien(s)
101 communes ont 2 bien(s)
152 communes ont 3 bien(s)
690 communes ont 4 bien(s)


##### **Loyers par département**

In [21]:
def rent_by_dep(df):
    print("="*100)
    print("PRIX DES LOYERS PAR M² PAR DÉPARTEMENT")
    print("="*100)

    fig = go.Figure()
    df = df.copy()
    df['departement_label'] = df['DEP'] + ' (' + df['nom_departement'] + ')'

    fig.add_trace(go.Box(
        y=df["loyer_predit_m2"],
        x=df["departement_label"],
        name="Loyer par département",
        hovertemplate=
        'Département: %{x}<br>' +
        'Loyer/m²: %{y:.2f}€<br>' +
        'Type de bien: %{customdata}<br>' +
        '<extra></extra>',
        customdata=df["type_local"]
    ))

    fig.update_layout(
        title="Distribution des loyers par département",
        xaxis_title="Département",
        yaxis_title="Loyer prédit par m² (€)",
    )

    fig.show()

rent_by_dep(filtered_df)

PRIX DES LOYERS PAR M² PAR DÉPARTEMENT


**Points clés :**

1. *Prix médians des loyers :*
    - Paris (75) présente le prix médian de location au m² le plus élevé à 31,15 €
    - Seine-et-Marne (77) a le prix médian de location au m² le plus bas à 14,26 €


2. *Variabilité des Prix :*
    - Certains départements comme Paris (75) et Yvelines (78) montrent une plus grande variabilité des prix de location (boîte plus large et moustaches plus longues)
    - Les départements comme l'Essonne (91) et le Val-d'Oise (95) présentent des distributions de prix plus compactes


3. *Valeurs Extrêmes :*
    - Plusieurs départements ont des valeurs extrêmes élevées (points au-dessus des moustaches), notamment dans les Hauts-de-Seine, le Val d'Oise et le Val-de-Marne


4. *Modèles Spatiaux :*
    - On observe un gradient de prix clair à travers les départements
    - Les départements centraux (Paris, Hauts-de-Seine) ont tendance à avoir des prix plus élevés
    - Les départements périphériques (Essonne, Val-d'Oise) présentent des loyers plus bas

---

##### **Loyers par type de bien**

In [22]:
def rent_by_type(df):
    print("="*100)
    print("PRIX DES LOYERS PAR M² SELON LE TYPE DE BIEN")
    print("="*100)
    
    fig = go.Figure()

    fig.add_trace(go.Box(
        y=df["loyer_predit_m2"],
        x=df["type_local"],
        name="Loyer prédit par m²",
        hovertemplate=
        'Type de bien: %{x}<br>' +
        'Loyer/m²: %{y:.2f}€<br>' +
        'Département: %{customdata}<br>' +
        '<extra></extra>',
        customdata=df["DEP"]
    ))

    fig.update_layout(
        title="Distribution des loyers par m² selon le type de bien",
        xaxis_title="Type de bien",
        yaxis_title="Loyer prédit par m² (€)",
    )

    fig.show()

rent_by_type(filtered_df)

PRIX DES LOYERS PAR M² SELON LE TYPE DE BIEN


**Points clés :**

1. *Prix médians par type de bien :*
    - Appartements T1-T2 : prix médian le plus élevé
    - Maisons : prix médian le plus bas


2. *Variabilité des Prix :*
    - Généralement la distribution de prix relativement resserrée
    - Appartements T3+ : plus grande variabilité des prix (de 7.53€ à 37.17€)
    - Présence de nombreuses valeurs extrêmes (points au-dessus des moustaches)


3. *Valeurs Extrêmes :*
    - Tous les types de biens présentent des valeurs extrêmes
    - Les appartements montrent les valeurs extrêmes les plus élevées, dépassant 37 €/m²


4. *Implications :*
    - Les petits appartements (T1-T2) semblent avoir les loyers au m² les plus élevés
    - Les maisons présentent généralement des loyers plus compacts et plus bas au m²
    - La taille et le type de bien influencent significativement le prix au m²

##### **Matrice de corrélations ?**


In [23]:
def plot_correlation_matrix(df, numeric_columns=numeric_columns):
    print("="*100)
    print("MATRICE DE CORRÉLATION")
    print("="*100)

    correlation_matrix = df[numeric_columns].corr()

    fig = go.Figure(data=go.Heatmap(
        z=correlation_matrix.values,
        x=correlation_matrix.columns,
        y=correlation_matrix.index,
        text=correlation_matrix.values.round(2),
        texttemplate="%{text}",
        colorscale='RdBu',
        zmin=-1,
        zmax=1
    ))

    fig.update_layout(
        title="Matrice de corrélation",
        xaxis_nticks=36,
        height=500,
        width=500
    )
    fig.show()

# plot_correlation_matrix(filtered_df)


In [24]:
import plotly.graph_objs as go
import plotly.io as pio

def distribution_loyers(df, column):
    print("="*100)
    print("DISTRIBUTION DES LOYERS - ASYMÉTRIE ET KURTOSIS")
    print("="*100)

    fig = go.Figure(
        go.Histogram(
            x=df[column],
            name='Distribution',
            opacity=0.7,
            marker_color='#3498db'
        )
    )
    
    # Update layout
    fig.update_layout(
        title_text=f'Distribution de {column}',
        xaxis_title=column,
        yaxis_title='Fréquence',
        height=500,
        width=800
    )
    
    print(f"Asymétrie: {df[column].skew():.4f}")
    print(f"Kurtosis: {df[column].kurtosis():.4f}")
    
    fig.show()

distribution_loyers(filtered_df, "loyer_predit_m2")

DISTRIBUTION DES LOYERS - ASYMÉTRIE ET KURTOSIS
Asymétrie: 1.2628
Kurtosis: 3.0693


1. **Graphe**
    - Distribution asymétrique à droite : la queue s'étend vers la droite
    - Le pic est décalé vers la gauche
    - La plupart des prix de location sont concentrés dans la gamme médiane inférieure

2. **Asymétrie (1,2628)**
    - Significativement positive (> 0,5)
    - Indique de nombreux prix de location de faible valeur
    - Quelques locations haut de gamme "tirent" la distribution vers la droite

3. **Kurtosis (3,0693)**
    - Légèrement plus élevé qu'une distribution normale (3,0)
    - Suggère une distribution plus pointue
    - Plus concentrée autour des valeurs centrales
    - Moins d'valeurs extrêmes par rapport à une distribution parfaitement normale

4. **Implications Pratiques**
    - Les prix de location typiques se situent autour de 15-22€/m²
    - Quelques propriétés haut de gamme avec des prix supérieurs à 25€/m²
    - La plupart des propriétés ont des prix relativement cohérents
    - Opportunités potentielles d'investissement dans la gamme 15-20€/m²

## **Ajouter des info géographique par commune**

In [25]:
filtered_df = communes_df.merge(filtered_df, left_on='code_commune_INSEE', right_on='INSEE_COM')
filtered_df.drop(columns=['code_commune_INSEE', 'borne_inferieure_intervalle', 'borne_superieure_intervalle', 'type_prediction', 'coefficient_determination_ajuste'], inplace=True)

---

## Export des données nettoyées

### Sauvegarde du dataset final

In [26]:
# CODEZ ICI: Exporter le dataframe nettoyé
filtered_df.to_csv("data/cleaned/loyers_IDF_2024.csv", index=False)
print(f" Dataset nettoyé exporté : loyers_IDF_2024.csv - {filtered_df.shape[0]} lignes, {filtered_df.shape[1]} colonnes")

 Dataset nettoyé exporté : loyers_IDF_2024.csv - 3548 lignes, 12 colonnes


---

## Synthèse du nettoyage

### Résumé des transformations effectuées

<!-- COMPLÉTEZ ICI: Résumez toutes les étapes de nettoyage -->
<!-- 1. Données brutes initiales : X lignes -->
<!-- 2. Après suppression des valeurs manquantes : Y lignes -->
<!-- 3. Après filtrage des aberrations : Z lignes -->
<!-- 4. Variables créées : liste -->
<!-- 5. Données finales : N lignes, M colonnes -->

### Qualité des données finales

In [27]:
get_summary(filtered_df, show_missing=True)

RÉSUMÉ DU JEU DE DONNÉES
Dimensions : 3548 lignes × 12 colonnes

Types de données :
object     7
float64    3
int64      2
Name: count, dtype: int64

Valeurs manquantes :
  Aucune valeur manquante !


### Recommandations pour l'analyse

<!-- COMPLÉTEZ ICI: Notez les points importants pour l'analyse suivante -->
<!-- - Variables les plus pertinentes identifiées -->
<!-- - Limitations des données -->
<!-- - Suggestions pour les widgets -->

---

**Notebook préparé par :**
- Ashley OHNONA
- Harisoa RANDRIANASOLO
- Fairouz YOUDARENE
- Jennifer ZAHORA

**Date :** <!-- COMPLÉTEZ ICI: Date -->

**Dataset final :** `donnees_nettoyees.csv`